# QKD2: Complete the "Alice," "Bob," and "Alice & Bob" sections

In [ ]:
#@title #Execute This Cell First!
#This cell's contents should match those of photon.py

import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    def prepareR(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1j/np.sqrt(2), avgPhotonNumber)

    def prepareL(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1j/np.sqrt(2), avgPhotonNumber)

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def measureDA(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureRL(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a-b*1j)/np.sqrt(2)
        self.beta  = (a+b*1j)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "R"
        if outcome == "V": return "L"
        else: return outcome

    def applyPolarizer(self, theta, phi):
	# Apply a polarizing filter according to the input parameters.
	# theta=0,    phi=0:     H polarizer
	# theta=pi/2, phi=0:     V polarizer
	# theta=pi/4, phi=0:     D polarizer
	# theta=pi/4, phi=pi:    A polarizer
	# theta=pi/4, phi=+pi/2: R polarizer
	# theta=pi/4, phi=-pi/2: L polarizer
        z = np.exp(1j*phi)
        a = self.alpha
        b = self.beta
        self.alpha = a*(1+np.cos(2*theta))/2 + b*np.sin(2*theta)/2*np.conj(z)
        self.beta  = a*np.sin(2*theta)/2*z + b*(1-np.cos(2*theta))/2
        # Now add an extra vacuum component.
        vac = Photon()
        vac.prepareVacuum()
        a = vac.alpha
        b = vac.beta
        self.alpha = self.alpha + a*np.sin(theta)**2 + b*(-np.sin(2*theta)/2)*np.conj(z)
        self.beta  = self.beta  + a*(-np.sin(2*theta)/2)*z + b*np.cos(theta)**2

    def applyUnitaryGate(self, theta, phi, lamb):
        U = [[0,0],[0,0]]
        z1 = np.exp(1j*phi)
        z2 = -np.exp(1j*lamb)
        z3 = np.exp(1j*(lamb+phi))
        U[0][0] = np.cos(theta/2)
        U[1][0] = np.sin(theta/2)*z1
        U[0][1] = np.sin(theta/2)*z2
        U[1][1] = np.cos(theta/2)*z3
        a = self.alpha
        b = self.beta
        self.alpha = U[0][0]*a + U[0][1]*b
        self.beta  = U[1][0]*a + U[1][1]*b

    def applyXGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(np.pi, 0, np.pi)

    def applyYGate(self):
        # Applies the Pauli Y gate
        self.applyUnitaryGate(np.pi, np.pi/2, np.pi/2)

    def applyZGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(0, np.pi, 0)

    def applyHGate(self):
        # Applied the Hadamard (half-wavelength) gate
        self.applyUnitaryGate(np.pi/2, 0, np.pi)

    def applyQGate(self):
        # Applies the SH (quarter-wavelength) gate
        self.applyUnitaryGate(np.pi/2, np.pi/2, np.pi)

    def applyNoisyGate(self, p):
        # This operation acts as a depolarizing channel.
	# p = 0 leaves the photon unchanged.
	# p = 1 yields a completely random photon.
	# 0 < p < 1 yields a partially random photon.
        if p < 0 or p > 1:
            raise InputError
        theta = np.arccos(1 - 2*random.uniform(0,1)*p)
        phi   = p*(2*random.uniform(0,1) - 1)*np.pi
        lamb  = p*(2*random.uniform(0,1) - 1)*np.pi
        self.applyUnitaryGate(theta, phi, lamb)

    def applyAttenuation(self, r):
	# This operation acts as a partially reflecting beam splitter.
	# r = 0 leaves the photon unchanged.
	# r = 1 completely absorbs the photon, leaving a vacuum state.
	# 0 < r < 1 partially attenuates the photon and adds some vacuum.
	# r is the reflectivity.
        if r < 0 or r > 1:
            raise InputError
        t = np.sqrt(1-r*r) # t is the transmissivity.
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = (self.alpha)*t + (vac.alpha)*r
        self.beta  = (self.beta )*t + (vac.beta)*r

In [ ]:
n = 100 # number of photons

In [ ]:
# Alice --------------------------------------------

# Alice generates the raw key.
keyAlice = ""
for i in range(n): # Iterate over the number of photons.
    # Append a random character ('0' or '1') to the end.
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        keyAlice += '0'
    else:
        keyAlice += '1'

print(keyAlice)


0011101111110100010101011101111100100111101000100000010011110111110101001110110000110111001000001100


In [ ]:
# Alice chooses the encoding basis for each key bit.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisAlice = ""
# TODO: Put your code here.
for i in range(len(keyAlice)):
  if random.randint(0,1)==0:
    basisAlice += "+"
  else:
    basisAlice += "x"
print("basisAlice  = " + basisAlice)

basisAlice  = +xx++x+x+xxx+x++xx+x+x++++xxxx+x+x+++x+x+++x+xx+xx++x+xx++x++xx++xx++xxxx+x+xx+xxx+x+x++xxxxx++x++x+


In [ ]:
# Alice selects a photon state according to the key and basis.
# This should be a string of the characters 'H', 'V', 'D', 'A'.
photonAlice = ""
# TODO: Put your code here.
for i in range(len(basisAlice)):
  if basisAlice[i] == "+":
    if keyAlice[i] == "0":
      photonAlice += "H"
    elif keyAlice[i] == "1":
      photonAlice += "V"
  elif basisAlice[i] == "x":
    if keyAlice[i] == "0":
      photonAlice += "D"
    elif keyAlice[i] == "1":
      photonAlice += "A"
print("photonAlice  = " + photonAlice)

photonAlice  = HDAVVDVAVAAAHAHHDAHAHAHVVVDAAAVAHDVHHAVAVHVDHDAHDDHHDVDDVVAVHAAVVADVHADDAVAHAAHDDDVAHAVVDDADDHHDVVDH


In [ ]:
# Alice prepares and sends each photon.
# Use the methods of the Photon class to prepare each photon.
photonArray = [Photon() for i in range(n)]
# TODO: Put your code here.

for i in range(n):
  if photonAlice[i] == "H":
    photonArray[i].prepareH(n)
  elif photonAlice[i] == "V":
    photonArray[i].prepareV(n)
  elif photonAlice[i] == "D":
    photonArray[i].prepareD(n)
  elif photonAlice[i] == "A":
    photonArray[i].prepareA(n)
  # print(photonArray)

In [ ]:
# Eve   --------------------------------------------

# You should implement this section after completing Alice and Bob.
# Eve is allowed to do whatever she wants to the photonAlice array.
# She cannot, however, have knowledge of Alice's or Bob's choice of bases,
# nor Bob's measurement outcomes, until they are publicly announced.

In [ ]:
# Eve selects a subsample of photons from Alice to measure.
# interceptIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=intercepted
interceptIndex = ""
sampleProp = 1
for i in range (n):
  if random.random() <= sampleProp:
    interceptIndex += "1";
  else:
    interceptIndex += "0";
print(interceptIndex)

# TODO: Put your code here.

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111


In [ ]:
# Eve chooses a basis to measure each intercepted photon.
# basisEve should be a string of n characters.
# Use the convention '+'=H/V, 'x'=D/A, ' '=not measured
basisEve = ""
for i in range(n):
  if interceptIndex[i] == "1":
    basisEve += "+" #No risk of zero percent correct
  else:
    basisEve += " "
print(basisEve)
# TODO: Put your code here.

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [ ]:
# Eve performs a measurement on each photon.
# outcomeEve should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeEve = ""
probDarkCountEve = 0.5 ##
for i in range(n):
  if basisEve[i] == '+':
    outcomeEve += Photon.measureHV(photonArray[i], probDarkCountEve)
  elif basisEve[i] == 'x':
    outcomeEve += Photon.measureDA(photonArray[i], probDarkCountEve)
  else:
    outcomeEve += " "
print(outcomeEve)

# TODO: Put your code here.

MMMMMMVMVMMMHMHHMMHMHMHVVMMMMMMMHMVHHMMMVHVMHMMMMMMHMVMMVMMVHMMVMMMVHMMMMVMMMMHMMMVMHMVVMMMMMHHMMMMH


In [ ]:
# Eve resends photons to Bob.
# Be sure to handle the cases in which Eve gets an invalid measurement.
probDA = 1
avgPhotonEve = n
for i in range(n):
  if outcomeEve[i] == "M" or outcomeEve[i] == "N":
    if random.random() <= probDA:
      photonArray[i].prepareD(avgPhotonEve)
    else:
      photonArray[i].prepareA(avgPhotonEve) #assuming M or N is DA basis
  elif outcomeEve[i] == "H":
    photonArray[i].prepareH(avgPhotonEve)
  elif outcomeEve[i] == "V":
    photonArray[i].prepareV(avgPhotonEve)




# TODO: Put your code here.

In [ ]:
# OPTIONAL: Put any other nasty tricks here.

In [ ]:
# Bob   --------------------------------------------

# Bob chooses a basis to measure each photon.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisBob = ""
# TODO: Put your code here.
for i in range(n):
  if random.randint(0,1)==0:
    basisBob += "+"
  else:
    basisBob += "x"

print("basisBob    = " + basisBob)

basisBob    = ++x+xxxx++xx+x++xx+xx++++xxxxx++++++xx++xx+x+x+x++x+xx+++++++++xx++xxx+x+++++xxxxxxxxx+xxxxx+xxxxx+x


In [ ]:
# Bob performs a measurement on each photon.
# Use the methods of the Photon class to measure each photon.
# outcomeBob should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeBob = ""
# TODO: Put your code here.

probDarkCount = 0.15

for i in range(n):
  if basisBob[i] == "+":
    outcomeBob += photonArray[i].measureHV(probDarkCount)
  elif basisBob[i] == "x":
    outcomeBob += photonArray[i].measureDA(probDarkCount)


print("outcomeBob  = " + outcomeBob)
print(len(outcomeBob))

outcomeBob  = MMDMDDMDVMDDHMHHDDHDMMMVVDDDDDMMHMVHMDMMMMVDHDMDMMDHDMMMVMMVMMMMDMMMMDMDMVMMMDMDDDMDMDVMDDDDMMMDDDMM
100


In [ ]:
# Bob infers the raw key.
# keyBob should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement
keyBob = ""
# TODO: Put your code here.

for i in range(n):
  if outcomeBob[i] == "H":
    keyBob += "0"
  elif outcomeBob[i] == "V":
    keyBob += "1"
  elif outcomeBob[i] == 'D':
    keyBob += "0"
  elif outcomeBob[i] == "A":
    keyBob += "1"
  elif outcomeBob[i] == "N" or outcomeBob[i] == 'M': #changed this
    keyBob += "-"

print("keyBob      = " + keyBob)
print(len(keyBob))


keyBob      = --0-00-01-000-000000---1100000--0-10-0----1000-0--000---1--1----0----0-0-1---0-000-0-01-0000---000--
100


In [ ]:
# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# Bob also announces which of his measurements were invalid.
# -----------------------------------------------------------

In [ ]:
# Alice & Bob ----------------------------------------------------------

# Alice and Bob extract their sifted keys.
# siftedAlice and siftedBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.

for i in range(n):
  if basisAlice[i] == basisBob[i]:
    siftedAlice += keyAlice[i]
    siftedBob += keyBob[i]
  else: #changed this
    siftedAlice += " "
    siftedBob += " "

print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

siftedAlice = 0 11 0 11 1101000101  011 01111 0 10 11   1000     00   11 10        1 0 1 0 1 000 1 11 0010   0    
siftedBob   = - 0- 0 01 000-000000  -11 0000- 0 10 0-   1000     00   1- 1-        0 0 1 - 0 000 0 01 0000   0    


In [ ]:
# Alice and Bob use a portion of their sifted keys to estimate the quantum bit error rate (QBER).
# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampleIndex = ""
sampledBobQBER = 0
# TODO: Put your code here.

sample_rate = 0.2  # Sample 20% of the sifted key for QBER estimation
for i in range(len(siftedAlice)):
    if random.random() < sample_rate:
        sampleIndex += "1"
    else:
        sampleIndex += "0"


error_count = 0
sampled_count = 0
for i in range(len(siftedAlice)):
    if sampleIndex[i] == "1":  # Check if the bit is sampled
        sampled_count += 1
        if siftedAlice[i] != siftedBob[i]:  # Check for mismatch
            error_count += 1

if sampled_count > 0:
    sampledBobQBER = error_count / sampled_count
else:
    sampledBobQBER = 0

print(sampleIndex)
print(sampledBobQBER)





0000001000100000000001100001010000000100001100100000011000100011011000000000000101000001001100000000
0.2727272727272727


In [ ]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""
# TODO: Put your code here.

for i in range(len(siftedAlice)):
  if sampleIndex[i] == "0":
    secureAlice += siftedAlice[i]
    secureBob += siftedBob[i]
  else:
    secureAlice += " " #fixed this
    secureBob += " "

print(secureAlice)
print(len(secureAlice))
print(secureBob)

0 11 0 11  101000101   11 0 1 1 0 10  1     00     00   11 10        1 0 1 0 1  0  1 11 00     0    
100
- 0- 0 01  00-000000   11 0 0 - 0 10  -     00     00   1- 1-        0 0 1 - 0  0  0 01 00     0    


In [ ]:
# Alice and Bob make a hard determination whether the channel is secure.
# If it looks like there's something fishy, better hit the kill switch!
channelSecure = True # default value, to be changed to False if Eve suspected
# TODO: Put your code here.




In [ ]:
# Eve ------------------------------------------------------------------

In [ ]:
# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured
keyEve = ""
# TODO: Put your code here.

for i in range(n):
  if outcomeEve[i] == "H":
    keyEve += "0"
  elif outcomeEve[i] == "V":
    keyEve += "1"
  elif outcomeEve[i] == 'D':
    keyEve += "0"
  elif outcomeEve[i] == "A":
    keyEve += "1"
  elif outcomeEve[i] == "N" or outcomeEve[i] == 'M': # should we assume its D?
    keyEve += "0"
  else:
    keyEve += " "

print("keyEve      = " + keyEve)
print(len(keyEve))

keyEve      = 0000001010000000000000011000000000100000101000000000010010010001000100000100000000100011000000000000
100


In [ ]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publically revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the '0', '1', ' '=removed
stolenEve = ""
for i in range(n):
  if outcomeEve[i] == photonAlice[i] and secureAlice[i] != " ":
    stolenEve += keyEve[i]
  else:
    stolenEve += " "

print(stolenEve)
print(len(stolenEve))

# TODO: Put your code here.

        1   0 00  0    11       0 10        0      0    1  10            1            1             
100


In [ ]:
# ANALYSIS -------------------------------------------------------------

# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
       secureKeyLengthBob += 1
       if secureAlice[i] == secureBob[i]:
           numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1-actualBobQBER) * secureKeyLengthBob / n * 1e6

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
       stolenKeyLengthEve += 1
       if secureAlice[i] == stolenEve[i]:
           numMatchEve += 1
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1-actualEveQBER) * stolenKeyLengthEve / n * 1e6


# DISPLAY RESULTS ------------------------------------------------------

print("")
print("basisAlice  = " + basisAlice)
print("basisBob    = " + basisBob)
print("basisEve    = " + basisEve)
print("")
print("keyAlice    = " + keyAlice)
print("keyBob      = " + keyBob)
print("keyEve      = " + keyEve)
print("")
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)
print("")
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)
print("stolenEve   = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob/1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve/1000) + " kbps")

# Your goal is to maximize secureKeyRateBob and minimize stolenKeyRateEve.


basisAlice  = +xx++x+x+xxx+x++xx+x+x++++xxxx+x+x+++x+x+++x+xx+xx++x+xx++x++xx++xx++xxxx+x+xx+xxx+x+x++xxxxx++x++x+
basisBob    = ++x+xxxx++xx+x++xx+xx++++xxxxx++++++xx++xx+x+x+x++x+xx+++++++++xx++xxx+x+++++xxxxxxxxx+xxxxx+xxxxx+x
basisEve    = ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

keyAlice    = 0011101111110100010101011101111100100111101000100000010011110111110101001110110000110111001000001100
keyBob      = --0-00-01-000-000000---1100000--0-10-0----1000-0--000---1--1----0----0-0-1---0-000-0-01-0000---000--
keyEve      = 0000001010000000000000011000000000100000101000000000010010010001000100000100000000100011000000000000

siftedAlice = 0 11 0 11 1101000101  011 01111 0 10 11   1000     00   11 10        1 0 1 0 1 000 1 11 0010   0    
siftedBob   = - 0- 0 01 000-000000  -11 0000- 0 10 0-   1000     00   1- 1-        0 0 1 - 0 000 0 01 0000   0    

secureAlice = 0 11 0 11  101000101   11 0 1 1 0 10  1     00     00   11 10 